In [ ]:
# -------------------------------------------------------------------------------------
# Dashboard de Análisis de Estrategias v2.3 - Versión Final Corregida
# -------------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import time

# ======================================================================================
# CONFIGURACIÓN
# ======================================================================================

class Config:
    """Configuración centralizada del dashboard"""
    SCATTER_HEIGHT = 600
    SCATTER_WIDTH = 550
    HEATMAP_HEIGHT = 700
    HEATMAP_WIDTH = 1100
    PNL_HEIGHT = 500
    PNL_WIDTH = 1100
    HOURLY_PROFILE_HEIGHT = 450
    HOURLY_PROFILE_WIDTH = 1100
    DD_ANALYSIS_HEIGHT = 400
    DD_ANALYSIS_WIDTH = 550
    
    ANALYSIS_FILE = 'analisis_final_automatizado.xlsx'
    TRADES_FILE = 'operaciones_maestro_combinado_DEC23TOSEP25.csv'
    
    TABLE_STYLES = [
        {'selector': 'th', 'props': [('background-color', '#333'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('color', 'white'), ('border-color', '#555')]},
        {'selector': '', 'props': [('background-color', '#222')]}
    ]
    
    CURVE_COLOR = '#7C3AED'
    HOURLY_PROFIT_COLOR = '#00CF9B'
    HOURLY_LOSS_COLOR = '#FF6B35'
    
    # Configuración de análisis de Drawdowns
    DD_THRESHOLD = 400  # Umbral para DDs significativos (EDITABLE)

# ======================================================================================
# CARGA DE DATOS
# ======================================================================================

def cargar_datos():
    """Carga y valida los archivos de datos necesarios"""
    try:
        df_results = pd.read_excel(Config.ANALYSIS_FILE)
        df_trades = pd.read_csv(Config.TRADES_FILE)
        df_trades['timestamp'] = pd.to_datetime(df_trades['timestamp'])
        df_trades['hora'] = df_trades['timestamp'].dt.time
        
        print(f"✓ Datos cargados: {len(df_results)} estrategias, {len(df_trades)} operaciones")
        return df_results, df_trades
    except FileNotFoundError as e:
        print(f"✗ Error: No se encontró el archivo {e.filename}")
        return None, None
    except Exception as e:
        print(f"✗ Error al cargar datos: {str(e)}")
        return None, None

# ======================================================================================
# FUNCIONES DE ANÁLISIS
# ======================================================================================

def calcular_perfil_horario(stop_loss, take_profit, df_trades, direccion='Ambas'):
    """Calcula el perfil de rentabilidad por bloques de 30 minutos"""
    df_filtered = df_trades.copy()
    if direccion == 'Largo':
        df_filtered = df_filtered[df_filtered['direction'] == 'Largo']
    elif direccion == 'Corto':
        df_filtered = df_filtered[df_filtered['direction'] == 'Corto']
    
    if len(df_filtered) == 0:
        return {'horas': [], 'pnl': [], 'counts': []}
    
    block_pnl = {}
    block_counts = {}
    
    for idx, row in df_filtered.iterrows():
        recorrido_str = row['Recorrido']
        timestamp = row['timestamp']
        hora = timestamp.hour
        minuto = timestamp.minute
        bloque = f"{hora:02d}:{'00' if minuto < 30 else '30'}"
        
        if not isinstance(recorrido_str, str) or pd.isna(recorrido_str) or recorrido_str == '':
            resultado = -stop_loss
        else:
            resultado = -stop_loss
            eventos = recorrido_str.split(',')
            
            for evento in eventos:
                if not evento:
                    continue
                tipo = evento[0].upper()
                try:
                    valor = int(evento[1:])
                except (ValueError, IndexError):
                    continue
                if tipo == 'M' and valor >= take_profit:
                    resultado = take_profit
                    break
                elif tipo == 'D' and valor >= stop_loss:
                    resultado = -stop_loss
                    break
        
        if bloque not in block_pnl:
            block_pnl[bloque] = 0
            block_counts[bloque] = 0
        block_pnl[bloque] += resultado
        block_counts[bloque] += 1
    
    bloques_labels = []
    pnl_values = []
    count_values = []
    
    for hora in range(24):
        for minuto in ['00', '30']:
            bloque = f"{hora:02d}:{minuto}"
            bloques_labels.append(bloque)
            pnl_values.append(block_pnl.get(bloque, 0))
            count_values.append(block_counts.get(bloque, 0))
    
    return {'horas': bloques_labels, 'pnl': pnl_values, 'counts': count_values}


def analizar_drawdowns_detallado(capital_acumulado, timestamps):
    """
    Analiza todos los drawdowns de una curva de capital
    
    Returns:
        dict con análisis completo de drawdowns
    """
    print(f"DEBUG analizar_drawdowns: Iniciando con {len(capital_acumulado)} puntos de capital")
    
    if len(capital_acumulado) < 2:
        print("DEBUG: Capital acumulado muy corto")
        return {
            'top_drawdowns': [],
            'dd_promedio': 0,
            'dd_total_pips': 0,
            'num_dds': 0,
            'frecuencia_significativos': {'count': 0, 'avg_interval_days': 0, 'intervals': []},
            'tiempo_total_en_dd': 0,
            'porcentaje_en_dd': 0
        }
    
    running_max = np.maximum.accumulate(capital_acumulado)
    drawdown = running_max - capital_acumulado
    max_dd = np.max(drawdown)
    print(f"DEBUG: Max DD calculado: {max_dd:.1f} pips")
    
    # Identificar todos los periodos de drawdown
    drawdowns = []
    en_dd = False
    dd_inicio = None
    dd_max_profundidad = 0
    dd_max_idx = None
    
    for i in range(len(drawdown)):
        if drawdown[i] > 0:  # Estamos en DD
            if not en_dd:
                # Inicio de un nuevo DD
                en_dd = True
                dd_inicio = i
                dd_max_profundidad = drawdown[i]
                dd_max_idx = i
            else:
                # Ya estamos en DD, actualizar máximo si es necesario
                if drawdown[i] > dd_max_profundidad:
                    dd_max_profundidad = drawdown[i]
                    dd_max_idx = i
        else:  # drawdown[i] == 0, estamos en un nuevo máximo
            if en_dd:
                # Fin del DD (se recuperó completamente)
                dd_fin = i
                dd_duracion = dd_fin - dd_inicio
                
                # Timestamps
                inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
                fin_time = timestamps[dd_fin - 1] if dd_fin <= len(timestamps) else None
                max_time = timestamps[dd_max_idx] if dd_max_idx < len(timestamps) else None
                
                tiempo_real = None
                if inicio_time and fin_time:
                    tiempo_real = fin_time - inicio_time
                
                drawdowns.append({
                    'profundidad': dd_max_profundidad,
                    'duracion_ops': dd_duracion,
                    'inicio_idx': dd_inicio,
                    'max_idx': dd_max_idx,
                    'fin_idx': dd_fin,
                    'inicio_time': inicio_time,
                    'max_time': max_time,
                    'fin_time': fin_time,
                    'tiempo_real': tiempo_real,
                    'recuperado': True
                })
                print(f"DEBUG: DD recuperado encontrado - Profundidad: {dd_max_profundidad:.1f}, Duración: {dd_duracion} ops")
                
                # Reset para el siguiente DD
                en_dd = False
                dd_max_profundidad = 0
    
    # CRÍTICO: Si terminamos aún en DD, guardarlo como DD activo
    if en_dd and dd_max_profundidad > 0:
        dd_fin = len(capital_acumulado) - 1
        dd_duracion = dd_fin - dd_inicio
        inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
        max_time = timestamps[dd_max_idx] if dd_max_idx < len(timestamps) else None
        
        drawdowns.append({
            'profundidad': dd_max_profundidad,
            'duracion_ops': dd_duracion,
            'inicio_idx': dd_inicio,
            'max_idx': dd_max_idx,
            'fin_idx': None,
            'inicio_time': inicio_time,
            'max_time': max_time,
            'fin_time': None,
            'tiempo_real': None,
            'recuperado': False
        })
        print(f"DEBUG: DD activo (sin recuperar) encontrado - Profundidad: {dd_max_profundidad:.1f}, Duración: {dd_duracion} ops")
    
    # Si no encontramos ningún DD pero hay drawdown > 0, es porque nunca hubo recuperación
    # En este caso, crear un único DD que abarca todo el periodo en drawdown
    if len(drawdowns) == 0 and max_dd > 0:
        print(f"DEBUG: Caso especial - No se encontraron DDs pero max_dd = {max_dd:.1f}")
        max_dd_idx = np.argmax(drawdown)
        max_dd_value = drawdown[max_dd_idx]
        
        # Buscar donde empezó (último punto en 0 antes del máximo)
        dd_inicio = 0
        for i in range(max_dd_idx, -1, -1):
            if drawdown[i] == 0:
                dd_inicio = i
                break
        
        inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
        max_time = timestamps[max_dd_idx] if max_dd_idx < len(timestamps) else None
        
        drawdowns.append({
            'profundidad': max_dd_value,
            'duracion_ops': len(capital_acumulado) - dd_inicio - 1,
            'inicio_idx': dd_inicio,
            'max_idx': max_dd_idx,
            'fin_idx': None,
            'inicio_time': inicio_time,
            'max_time': max_time,
            'fin_time': None,
            'tiempo_real': None,
            'recuperado': False
        })
        print(f"DEBUG: DD único creado - Profundidad: {max_dd_value:.1f}, desde índice {dd_inicio}")
    
    print(f"DEBUG: Total DDs encontrados: {len(drawdowns)}")
    
    # Ordenar por profundidad
    drawdowns_sorted = sorted(drawdowns, key=lambda x: x['profundidad'], reverse=True)
    
    # Top 10
    top_10 = drawdowns_sorted[:10]
    
    # DD promedio
    dd_promedio = np.mean([dd['profundidad'] for dd in drawdowns]) if drawdowns else 0
    
    # Total pips en DD
    dd_total_pips = sum([dd['profundidad'] for dd in drawdowns])
    
    # Análisis de DDs significativos (>= threshold)
    dds_significativos = [dd for dd in drawdowns if dd['profundidad'] >= Config.DD_THRESHOLD]
    
    intervalos = []
    if len(dds_significativos) > 1:
        # Ordenar por tiempo de inicio
        dds_sig_sorted = sorted([dd for dd in dds_significativos if dd['inicio_time']], key=lambda x: x['inicio_time'])
        
        for i in range(1, len(dds_sig_sorted)):
            dd_anterior = dds_sig_sorted[i-1]
            dd_actual = dds_sig_sorted[i]
            
            # Solo calcular intervalo si el anterior se recuperó
            if dd_anterior.get('fin_time') and dd_actual.get('inicio_time'):
                intervalo = dd_actual['inicio_time'] - dd_anterior['fin_time']
                intervalos.append(intervalo.total_seconds() / 86400)  # En días
    
    avg_interval = np.mean(intervalos) if intervalos else 0
    
    # Tiempo total en DD (sumar todas las duraciones)
    tiempo_en_dd = sum([dd['duracion_ops'] for dd in drawdowns])
    porcentaje_en_dd = (tiempo_en_dd / (len(capital_acumulado) - 1) * 100) if len(capital_acumulado) > 1 else 0
    
    resultado = {
        'top_drawdowns': top_10,
        'dd_promedio': dd_promedio,
        'dd_total_pips': dd_total_pips,
        'num_dds': len(drawdowns),
        'frecuencia_significativos': {
            'count': len(dds_significativos),
            'avg_interval_days': avg_interval,
            'intervals': intervalos
        },
        'tiempo_en_dd_ops': tiempo_en_dd,
        'porcentaje_en_dd': porcentaje_en_dd
    }
    
    print(f"DEBUG: Resultado final - {len(drawdowns)} DDs, Promedio: {dd_promedio:.1f}, Total pips: {dd_total_pips:.1f}")
    return resultado
    """
    Analiza todos los drawdowns de una curva de capital
    
    Returns:
        dict con análisis completo de drawdowns
    """
    if len(capital_acumulado) < 2:
        return {
            'top_drawdowns': [],
            'dd_promedio': 0,
            'dd_total_pips': 0,
            'num_dds': 0,
            'frecuencia_significativos': {'count': 0, 'avg_interval_days': 0, 'intervals': []},
            'tiempo_total_en_dd': 0,
            'porcentaje_en_dd': 0
        }
    
    running_max = np.maximum.accumulate(capital_acumulado)
    drawdown = running_max - capital_acumulado
    
    # Identificar todos los periodos de drawdown
    drawdowns = []
    en_dd = False
    dd_inicio = None
    dd_max_profundidad = 0
    dd_max_idx = None
    
    for i in range(len(drawdown)):
        if drawdown[i] > 0:  # Estamos en DD
            if not en_dd:
                # Inicio de un nuevo DD
                en_dd = True
                dd_inicio = i
                dd_max_profundidad = drawdown[i]
                dd_max_idx = i
            else:
                # Ya estamos en DD, actualizar máximo si es necesario
                if drawdown[i] > dd_max_profundidad:
                    dd_max_profundidad = drawdown[i]
                    dd_max_idx = i
        else:  # drawdown[i] == 0, estamos en un nuevo máximo
            if en_dd:
                # Fin del DD (se recuperó completamente)
                dd_fin = i
                dd_duracion = dd_fin - dd_inicio
                
                # Timestamps
                inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
                fin_time = timestamps[dd_fin - 1] if dd_fin <= len(timestamps) else None
                max_time = timestamps[dd_max_idx] if dd_max_idx < len(timestamps) else None
                
                tiempo_real = None
                if inicio_time and fin_time:
                    tiempo_real = fin_time - inicio_time
                
                drawdowns.append({
                    'profundidad': dd_max_profundidad,
                    'duracion_ops': dd_duracion,
                    'inicio_idx': dd_inicio,
                    'max_idx': dd_max_idx,
                    'fin_idx': dd_fin,
                    'inicio_time': inicio_time,
                    'max_time': max_time,
                    'fin_time': fin_time,
                    'tiempo_real': tiempo_real,
                    'recuperado': True
                })
                
                # Reset para el siguiente DD
                en_dd = False
                dd_max_profundidad = 0
    
    # CRÍTICO: Si terminamos aún en DD, guardarlo como DD activo
    if en_dd and dd_max_profundidad > 0:
        dd_fin = len(capital_acumulado) - 1
        dd_duracion = dd_fin - dd_inicio
        inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
        max_time = timestamps[dd_max_idx] if dd_max_idx < len(timestamps) else None
        
        drawdowns.append({
            'profundidad': dd_max_profundidad,
            'duracion_ops': dd_duracion,
            'inicio_idx': dd_inicio,
            'max_idx': dd_max_idx,
            'fin_idx': None,
            'inicio_time': inicio_time,
            'max_time': max_time,
            'fin_time': None,
            'tiempo_real': None,
            'recuperado': False
        })
    
    # Si no encontramos ningún DD pero hay drawdown > 0, es porque nunca hubo recuperación
    # En este caso, crear un único DD que abarca todo el periodo en drawdown
    if len(drawdowns) == 0 and np.max(drawdown) > 0:
        max_dd_idx = np.argmax(drawdown)
        max_dd_value = drawdown[max_dd_idx]
        
        # Buscar donde empezó (último punto en 0 antes del máximo)
        dd_inicio = 0
        for i in range(max_dd_idx, -1, -1):
            if drawdown[i] == 0:
                dd_inicio = i
                break
        
        inicio_time = timestamps[dd_inicio] if dd_inicio < len(timestamps) else None
        max_time = timestamps[max_dd_idx] if max_dd_idx < len(timestamps) else None
        
        drawdowns.append({
            'profundidad': max_dd_value,
            'duracion_ops': len(capital_acumulado) - dd_inicio - 1,
            'inicio_idx': dd_inicio,
            'max_idx': max_dd_idx,
            'fin_idx': None,
            'inicio_time': inicio_time,
            'max_time': max_time,
            'fin_time': None,
            'tiempo_real': None,
            'recuperado': False
        })
    
    # Ordenar por profundidad
    drawdowns_sorted = sorted(drawdowns, key=lambda x: x['profundidad'], reverse=True)
    
    # Top 10
    top_10 = drawdowns_sorted[:10]
    
    # DD promedio
    dd_promedio = np.mean([dd['profundidad'] for dd in drawdowns]) if drawdowns else 0
    
    # Total pips en DD
    dd_total_pips = sum([dd['profundidad'] for dd in drawdowns])
    
    # Análisis de DDs significativos (>= threshold)
    dds_significativos = [dd for dd in drawdowns if dd['profundidad'] >= Config.DD_THRESHOLD]
    
    intervalos = []
    if len(dds_significativos) > 1:
        # Ordenar por tiempo de inicio
        dds_sig_sorted = sorted([dd for dd in dds_significativos if dd['inicio_time']], key=lambda x: x['inicio_time'])
        
        for i in range(1, len(dds_sig_sorted)):
            dd_anterior = dds_sig_sorted[i-1]
            dd_actual = dds_sig_sorted[i]
            
            # Solo calcular intervalo si el anterior se recuperó
            if dd_anterior.get('fin_time') and dd_actual.get('inicio_time'):
                intervalo = dd_actual['inicio_time'] - dd_anterior['fin_time']
                intervalos.append(intervalo.total_seconds() / 86400)  # En días
    
    avg_interval = np.mean(intervalos) if intervalos else 0
    
    # Tiempo total en DD (sumar todas las duraciones)
    tiempo_en_dd = sum([dd['duracion_ops'] for dd in drawdowns])
    porcentaje_en_dd = (tiempo_en_dd / (len(capital_acumulado) - 1) * 100) if len(capital_acumulado) > 1 else 0
    
    return {
        'top_drawdowns': top_10,
        'dd_promedio': dd_promedio,
        'dd_total_pips': dd_total_pips,
        'num_dds': len(drawdowns),
        'frecuencia_significativos': {
            'count': len(dds_significativos),
            'avg_interval_days': avg_interval,
            'intervals': intervalos
        },
        'tiempo_en_dd_ops': tiempo_en_dd,
        'porcentaje_en_dd': porcentaje_en_dd
    }


def simular_curva_capital(stop_loss, take_profit, df_trades, direccion='Ambas', hora_inicio=None, hora_fin=None):
    """Simula la curva de capital basada en los recorridos de precio"""
    df_filtered = df_trades.copy()
    if direccion == 'Largo':
        df_filtered = df_filtered[df_filtered['direction'] == 'Largo']
    elif direccion == 'Corto':
        df_filtered = df_filtered[df_filtered['direction'] == 'Corto']
    
    if hora_inicio is not None and hora_fin is not None:
        if hora_inicio <= hora_fin:
            df_filtered = df_filtered[(df_filtered['hora'] >= hora_inicio) & (df_filtered['hora'] <= hora_fin)]
        else:
            df_filtered = df_filtered[(df_filtered['hora'] >= hora_inicio) | (df_filtered['hora'] <= hora_fin)]
    
    if len(df_filtered) == 0:
        return {
            'x': np.array([0]), 'y': np.array([0]), 'num_trades': 0,
            'stats': {'ganadas': 0, 'perdidas': 0, 'win_rate': 0, 'avg_win': 0, 'avg_loss': 0, 'max_dd': 0, 'dd_duration': 0, 'dd_start': None, 'dd_end': None}
        }
    
    pnl_history = []
    ganadas = 0
    perdidas = 0
    sum_wins = 0
    sum_losses = 0
    timestamps = []
    
    for idx, row in df_filtered.iterrows():
        recorrido_str = row['Recorrido']
        timestamps.append(row['timestamp'])
        
        if not isinstance(recorrido_str, str) or pd.isna(recorrido_str) or recorrido_str == '':
            pnl_history.append(-stop_loss)
            perdidas += 1
            sum_losses += stop_loss
            continue
        
        resultado = -stop_loss
        eventos = recorrido_str.split(',')
        
        for evento in eventos:
            if not evento:
                continue
            tipo = evento[0].upper()
            try:
                valor = int(evento[1:])
            except (ValueError, IndexError):
                continue
            if tipo == 'M' and valor >= take_profit:
                resultado = take_profit
                break
            elif tipo == 'D' and valor >= stop_loss:
                resultado = -stop_loss
                break
        
        pnl_history.append(resultado)
        
        if resultado > 0:
            ganadas += 1
            sum_wins += resultado
        else:
            perdidas += 1
            sum_losses += abs(resultado)
    
    capital_acumulado = np.cumsum([0] + pnl_history)
    running_max = np.maximum.accumulate(capital_acumulado)
    drawdown = running_max - capital_acumulado
    
    # Encontrar el máximo drawdown y su duración
    max_drawdown = np.max(drawdown)
    max_dd_idx = np.argmax(drawdown)
    
    # Calcular duración del drawdown
    dd_duration = 0
    dd_start_idx = None
    dd_end_idx = None
    dd_start_time = None
    dd_end_time = None
    
    if max_drawdown > 0 and max_dd_idx > 0:
        # Encontrar cuando empezó el drawdown (último máximo antes del punto más bajo)
        for i in range(max_dd_idx, -1, -1):
            if drawdown[i] == 0:  # Estaba en un nuevo máximo
                dd_start_idx = i
                break
        
        # Encontrar cuando se recuperó (siguiente vez que llega a drawdown = 0)
        for i in range(max_dd_idx, len(drawdown)):
            if drawdown[i] == 0:  # Se recuperó
                dd_end_idx = i
                break
        
        # Si encontramos inicio y fin, calcular duración
        if dd_start_idx is not None and dd_end_idx is not None:
            dd_duration = dd_end_idx - dd_start_idx
            if dd_start_idx < len(timestamps):
                dd_start_time = timestamps[dd_start_idx]
            if dd_end_idx <= len(timestamps):
                dd_end_time = timestamps[dd_end_idx - 1] if dd_end_idx > 0 else None
        elif dd_start_idx is not None:
            # Aún no se ha recuperado
            dd_duration = len(capital_acumulado) - dd_start_idx - 1
            dd_end_idx = None
            if dd_start_idx < len(timestamps):
                dd_start_time = timestamps[dd_start_idx]
    
    total_trades = ganadas + perdidas
    win_rate = (ganadas / total_trades * 100) if total_trades > 0 else 0
    avg_win = (sum_wins / ganadas) if ganadas > 0 else 0
    avg_loss = (sum_losses / perdidas) if perdidas > 0 else 0
    
    return {
        'x': np.arange(len(capital_acumulado)),
        'y': capital_acumulado,
        'num_trades': len(pnl_history),
        'stats': {
            'ganadas': ganadas,
            'perdidas': perdidas,
            'win_rate': win_rate,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'max_dd': max_drawdown,
            'dd_duration': dd_duration,
            'dd_start': dd_start_time,
            'dd_end': dd_end_time,
            'dd_recovered': dd_end_idx is not None
        }
    }

# ======================================================================================
# CLASE PRINCIPAL DEL DASHBOARD
# ======================================================================================

class TradingDashboard:
    """Dashboard interactivo para análisis de estrategias de trading"""
    
    def __init__(self, df_results, df_trades):
        self.df_original = df_results
        self.df_trades = df_trades
        self.df_display = df_results.copy()
        self.current_strategy = None
        
        self._crear_widgets()
        self._crear_graficos()
        self._crear_controles_curva()
        self._configurar_callbacks()
    
    def _crear_widgets(self):
        """Crea todos los widgets de control"""
        self.sort_column = widgets.Dropdown(
            options=list(self.df_original.columns),
            value='Composite_Score',
            description='Ordenar por:',
            style={'description_width': 'initial'}
        )
        
        self.sort_order = widgets.RadioButtons(
            options=['Descendente', 'Ascendente'],
            value='Descendente',
            description='Orden:'
        )
        
        self.filter_widgets = {}
        numeric_cols = self.df_original.select_dtypes(include=['number']).columns
        
        for col in numeric_cols:
            min_val = float(self.df_original[col].min())
            max_val = float(self.df_original[col].max())
            
            if min_val == max_val:
                continue
            
            is_integer = pd.api.types.is_integer_dtype(self.df_original[col].dtype)
            slider_class = widgets.IntRangeSlider if is_integer else widgets.FloatRangeSlider
            step = 1 if is_integer else (max_val - min_val) / 100
            
            self.filter_widgets[col] = slider_class(
                value=[min_val, max_val],
                min=min_val,
                max=max_val,
                step=step,
                description=f'{col}:',
                layout={'width': '800px'},
                readout_format='.2f' if not is_integer else '.0f',
                continuous_update=False,
                style={'description_width': '200px'}
            )
        
        self.table_output = widgets.Output()
    
    def _crear_controles_curva(self):
        """Crea los controles específicos para la curva de capital"""
        self.direction_selector = widgets.RadioButtons(
            options=['Ambas', 'Largo', 'Corto'],
            value='Ambas',
            description='Dirección:',
            style={'description_width': 'initial'}
        )
        
        self.time_filter_enabled = widgets.Checkbox(
            value=False,
            description='Filtrar por horario',
            style={'description_width': 'initial'}
        )
        
        self.time_range_slider = widgets.IntRangeSlider(
            value=[0, 1440],
            min=0,
            max=1440,
            step=30,
            description='Rango:',
            disabled=True,
            orientation='horizontal',
            readout=False,
            layout={'width': '600px'},
            style={'description_width': '60px'}
        )
        
        self.time_range_label = widgets.HTML(
            value='<span style="color: #999; font-size: 0.9em;">00:00 - 24:00 (Todo el día)</span>'
        )
        
        self.stats_label = widgets.HTML(
            value='<i style="color: #999;">Selecciona una estrategia para ver estadísticas</i>'
        )
        
        self.direction_selector.observe(self._on_curve_control_change, names='value')
        self.time_filter_enabled.observe(self._on_time_filter_toggle, names='value')
        self.time_range_slider.observe(self._on_time_slider_change, names='value')
    
    def _minutos_a_hora(self, minutos):
        """Convierte minutos desde medianoche a formato HH:MM"""
        if minutos >= 1440:
            return "24:00"
        horas = minutos // 60
        mins = minutos % 60
        return f"{horas:02d}:{mins:02d}"
    
    def _on_time_slider_change(self, change):
        """Callback cuando cambia el slider de tiempo"""
        inicio_min, fin_min = self.time_range_slider.value
        hora_inicio = self._minutos_a_hora(inicio_min)
        hora_fin = self._minutos_a_hora(fin_min)
        
        self.time_range_label.value = (
            f'<span style="color: #7C3AED; font-weight: bold; font-size: 0.95em;">'
            f'🕐 {hora_inicio} - {hora_fin}</span>'
        )
        
        if self.current_strategy is not None:
            self._actualizar_curva_capital(
                self.current_strategy['stop'],
                self.current_strategy['profit']
            )
    
    def _on_time_filter_toggle(self, change):
        """Callback cuando se activa/desactiva el filtro de tiempo"""
        enabled = change['new']
        self.time_range_slider.disabled = not enabled
        
        if enabled:
            inicio_min, fin_min = self.time_range_slider.value
            hora_inicio = self._minutos_a_hora(inicio_min)
            hora_fin = self._minutos_a_hora(fin_min)
            self.time_range_label.value = (
                f'<span style="color: #7C3AED; font-weight: bold; font-size: 0.95em;">'
                f'🕐 {hora_inicio} - {hora_fin}</span>'
            )
        else:
            self.time_range_label.value = (
                '<span style="color: #999; font-size: 0.9em;">00:00 - 24:00 (Todo el día)</span>'
            )
        
        if self.current_strategy is not None:
            self._actualizar_curva_capital(
                self.current_strategy['stop'],
                self.current_strategy['profit']
            )
    
    def _crear_graficos(self):
        """Crea todos los gráficos interactivos"""
        self.g_scatter_pf = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.SCATTER_HEIGHT,
                'width': Config.SCATTER_WIDTH,
                'title': 'Análisis R/R (Color = Profit Factor)',
                'xaxis_title': 'Max Drawdown',
                'yaxis_title': 'Net P/L'
            }
        )
        
        self.g_scatter_cs = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.SCATTER_HEIGHT,
                'width': Config.SCATTER_WIDTH,
                'title': 'Análisis R/R (Color = Composite Score)',
                'xaxis_title': 'Max Drawdown',
                'yaxis_title': 'Net P/L'
            }
        )
        
        self.g_heatmap = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.HEATMAP_HEIGHT,
                'width': Config.HEATMAP_WIDTH,
                'title': 'Mapa de Calor (Color = Recovery Factor)',
                'xaxis_title': 'Take Profit',
                'yaxis_title': 'Stop Loss'
            }
        )
        
        self.g_hourly_profile = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.HOURLY_PROFILE_HEIGHT,
                'width': Config.HOURLY_PROFILE_WIDTH,
                'title': 'Perfil de Rentabilidad por Bloques de 30 Minutos - Selecciona una estrategia',
                'xaxis_title': 'Hora del Día (bloques de 30 min)',
                'yaxis_title': 'P/L Neto (pips)',
                'xaxis': {'tickangle': -90, 'tickmode': 'linear', 'tickfont': {'size': 8}},
                'yaxis': {'zeroline': True, 'zerolinewidth': 2, 'zerolinecolor': '#555'},
                'bargap': 0.05,
                'showlegend': False
            }
        )
        
        self.g_pnl_curve = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.PNL_HEIGHT,
                'width': Config.PNL_WIDTH,
                'title': 'Curva de Capital - Selecciona una estrategia',
                'xaxis_title': 'Número de Operación',
                'yaxis_title': 'Capital Acumulado (pips)'
            }
        )
        
        # Gráfico de Top Drawdowns
        self.g_dd_top = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.DD_ANALYSIS_HEIGHT,
                'width': Config.DD_ANALYSIS_WIDTH,
                'title': 'Top 10 Drawdowns por Profundidad',
                'xaxis_title': 'Ranking',
                'yaxis_title': 'Profundidad (pips)',
                'showlegend': False
            }
        )
        
        # Gráfico de Distribución de DDs
        self.g_dd_dist = go.FigureWidget(
            layout={
                'template': 'plotly_dark',
                'height': Config.DD_ANALYSIS_HEIGHT,
                'width': Config.DD_ANALYSIS_WIDTH,
                'title': 'Distribución de Drawdowns',
                'xaxis_title': 'Profundidad (pips)',
                'yaxis_title': 'Frecuencia',
                'showlegend': False
            }
        )
        
        # Output para estadísticas de DD
        self.dd_stats_output = widgets.Output()
    
    def _configurar_callbacks(self):
        """Configura todos los callbacks de interactividad"""
        for widget in [self.sort_column, self.sort_order, *self.filter_widgets.values()]:
            widget.observe(self._on_filter_change, names='value')
    
    def _on_filter_change(self, change):
        """Callback cuando cambia algún filtro o criterio de ordenamiento"""
        df_filtered = self.df_original.copy()
        
        for col, slider in self.filter_widgets.items():
            min_val, max_val = slider.value
            df_filtered = df_filtered[(df_filtered[col] >= min_val) & (df_filtered[col] <= max_val)]
        
        ascending = (self.sort_order.value == 'Ascendente')
        self.df_display = df_filtered.sort_values(by=self.sort_column.value, ascending=ascending).reset_index(drop=True)
        
        self._actualizar_tabla()
        self._actualizar_visualizaciones()
    
    def _actualizar_tabla(self):
        """Actualiza la tabla de resultados"""
        with self.table_output:
            clear_output(wait=True)
            if len(self.df_display) == 0:
                print("No hay estrategias que cumplan los criterios actuales")
            else:
                display(self.df_display.head(10).style.set_table_styles(Config.TABLE_STYLES).format(precision=2))
    
    def _actualizar_visualizaciones(self):
        """Actualiza todos los gráficos con los datos filtrados"""
        if len(self.df_display) == 0:
            self.g_scatter_pf.data = []
            self.g_scatter_cs.data = []
            self.g_heatmap.data = []
            return
        
        hovertemplate = (
            "<b>Stop: %{customdata[0]}, Profit: %{customdata[1]}</b><br>"
            "Net P/L: %{y:.0f}<br>Max DD: %{x:.0f}<br>"
            "Win Rate: %{customdata[2]:.2f}%<br>PF: %{customdata[3]:.2f}<br>"
            "Score: %{customdata[4]:.2f}<extra></extra>"
        )
        
        custom_data = self.df_display[['Stop', 'Profit', 'Win Rate (%)', 'Profit Factor', 'Composite_Score']].values
        
        with self.g_scatter_pf.batch_update():
            self.g_scatter_pf.data = []
            scatter_pf = go.Scatter(
                x=self.df_display['Max Drawdown'],
                y=self.df_display['Net P/L'],
                mode='markers',
                marker=dict(color=self.df_display['Profit Factor'], colorscale='Plasma', showscale=True, size=8, colorbar=dict(title='Profit Factor')),
                customdata=custom_data,
                hovertemplate=hovertemplate
            )
            self.g_scatter_pf.add_trace(scatter_pf)
            self.g_scatter_pf.data[0].on_click(self._on_scatter_click)
        
        with self.g_scatter_cs.batch_update():
            self.g_scatter_cs.data = []
            scatter_cs = go.Scatter(
                x=self.df_display['Max Drawdown'],
                y=self.df_display['Net P/L'],
                mode='markers',
                marker=dict(color=self.df_display['Composite_Score'], colorscale='Viridis', showscale=True, size=8, colorbar=dict(title='Composite Score')),
                customdata=custom_data,
                hovertemplate=hovertemplate
            )
            self.g_scatter_cs.add_trace(scatter_cs)
            self.g_scatter_cs.data[0].on_click(self._on_scatter_click)
        
        with self.g_heatmap.batch_update():
            self.g_heatmap.data = []
            pivot_data = self.df_display.pivot_table(index='Stop', columns='Profit', values='Recovery Factor').fillna(0)
            heatmap = go.Heatmap(
                z=pivot_data.values,
                x=pivot_data.columns,
                y=pivot_data.index,
                colorscale='Viridis',
                colorbar=dict(title='Recovery Factor'),
                hovertemplate="TP: %{x}<br>SL: %{y}<br>RF: %{z:.2f}<extra></extra>"
            )
            self.g_heatmap.add_trace(heatmap)
            self.g_heatmap.data[0].on_click(self._on_heatmap_click)
    
    def _on_scatter_click(self, trace, points, state):
        """Callback cuando se hace clic en un punto del scatter plot"""
        if not points.point_inds:
            return
        idx = points.point_inds[0]
        selected = self.df_display.iloc[idx]
        stop = int(selected['Stop'])
        profit = int(selected['Profit'])
        self.current_strategy = {'stop': stop, 'profit': profit}
        self._actualizar_curva_capital(stop, profit)
    
    def _on_heatmap_click(self, trace, points, state):
        """Callback cuando se hace clic en una celda del heatmap"""
        if not (hasattr(points, 'xs') and hasattr(points, 'ys')):
            return
        profit = int(points.xs[0])
        stop = int(points.ys[0])
        self.current_strategy = {'stop': stop, 'profit': profit}
        self._actualizar_curva_capital(stop, profit)
    
    def _on_curve_control_change(self, change):
        """Callback cuando cambia dirección"""
        if self.current_strategy is not None:
            self._actualizar_curva_capital(self.current_strategy['stop'], self.current_strategy['profit'])
    
    def _actualizar_curva_capital(self, stop, profit):
        """Actualiza el gráfico de curva de capital y el perfil horario"""
        try:
            print(f"\n{'='*60}")
            print(f"🔍 INICIO - Analizando estrategia SL={stop}, TP={profit}")
            print(f"{'='*60}")
            
            direccion = self.direction_selector.value
            hora_inicio = None
            hora_fin = None
            
            if self.time_filter_enabled.value:
                inicio_min, fin_min = self.time_range_slider.value
                hora_inicio = time(inicio_min // 60, inicio_min % 60)
                hora_fin = time(fin_min // 60, fin_min % 60) if fin_min < 1440 else time(23, 59)
            
            print(f"📊 Simulando curva de capital...")
            curve_data = simular_curva_capital(stop, profit, self.df_trades, direccion, hora_inicio, hora_fin)
            print(f"✓ Curva simulada: {curve_data['num_trades']} operaciones")
            
            print(f"🕐 Calculando perfil horario...")
            hourly_data = calcular_perfil_horario(stop, profit, self.df_trades, direccion)
            print(f"✓ Perfil calculado: {len(hourly_data['horas'])} bloques")
            
            if curve_data['num_trades'] == 0:
                mensaje = f'⚠️ No hay operaciones'
                if direccion != 'Ambas':
                    mensaje += f' en {direccion}'
                if self.time_filter_enabled.value:
                    inicio_min, fin_min = self.time_range_slider.value
                    mensaje += f' entre {self._minutos_a_hora(inicio_min)} y {self._minutos_a_hora(fin_min)}'
                print(f"⚠️ {mensaje}")
                self.stats_label.value = f'<span style="color: #FF6B35;">{mensaje}</span>'
                with self.g_pnl_curve.batch_update():
                    self.g_pnl_curve.data = []
                    self.g_pnl_curve.layout.title = f'Sin operaciones para SL={stop}, TP={profit}'
                with self.g_hourly_profile.batch_update():
                    self.g_hourly_profile.data = []
                    self.g_hourly_profile.layout.title = 'Perfil de Rentabilidad - Sin datos'
                with self.g_dd_top.batch_update():
                    self.g_dd_top.data = []
                with self.g_dd_dist.batch_update():
                    self.g_dd_dist.data = []
                with self.dd_stats_output:
                    clear_output(wait=True)
                    display(widgets.HTML('<span style="color: #999;">Sin datos para análisis de drawdowns</span>'))
                return
            
            # Analizar drawdowns
            print(f"\n📉 Analizando drawdowns...")
            print(f"   - Capital acumulado: {len(curve_data['y'])} puntos")
            print(f"   - Timestamps disponibles: {len(curve_data.get('timestamps', []))}")
            
            if 'timestamps' in curve_data and len(curve_data['timestamps']) > 0:
                max_dd_calculado = np.max(np.maximum.accumulate(curve_data['y']) - curve_data['y'])
                print(f"   - Max DD en curva: {max_dd_calculado:.1f} pips")
                
                dd_analysis = analizar_drawdowns_detallado(curve_data['y'], curve_data['timestamps'])
                print(f"✓ Análisis completado: {dd_analysis['num_dds']} DDs encontrados")
            else:
                print(f"⚠️ No hay timestamps disponibles para análisis DD")
                dd_analysis = {
                    'top_drawdowns': [],
                    'dd_promedio': 0,
                    'dd_total_pips': 0,
                    'num_dds': 0,
                    'frecuencia_significativos': {'count': 0, 'avg_interval_days': 0, 'intervals': []},
                    'tiempo_en_dd_ops': 0,
                    'porcentaje_en_dd': 0
                }
            
            # Actualizar curva de capital
            print(f"\n📈 Actualizando gráfico de curva de capital...")
            with self.g_pnl_curve.batch_update():
                self.g_pnl_curve.data = []
                self.g_pnl_curve.add_scatter(
                    x=curve_data['x'],
                    y=curve_data['y'],
                    mode='lines',
                    line=dict(color=Config.CURVE_COLOR, width=2.5),
                    name=f'SL={stop}, TP={profit}'
                )
                final_pnl = curve_data['y'][-1]
                num_trades = curve_data['num_trades']
                direccion_emoji = {'Ambas': '↕️', 'Largo': '📈', 'Corto': '📉'}
                titulo = f'{direccion_emoji[direccion]} Curva de Capital: Stop={stop}, Profit={profit} | P/L Final: {final_pnl:.0f} pips | Operaciones: {num_trades}'
                if self.time_filter_enabled.value:
                    inicio_min, fin_min = self.time_range_slider.value
                    titulo += f' | 🕐 {self._minutos_a_hora(inicio_min)} - {self._minutos_a_hora(fin_min)}'
                self.g_pnl_curve.layout.title = titulo
            
            # Actualizar perfil horario
            print(f"🕐 Actualizando perfil horario...")
            with self.g_hourly_profile.batch_update():
                self.g_hourly_profile.data = []
                if len(hourly_data['horas']) > 0:
                    colors = [Config.HOURLY_PROFIT_COLOR if pnl >= 0 else Config.HOURLY_LOSS_COLOR for pnl in hourly_data['pnl']]
                    hover_texts = [f"<b>{hora}</b><br>P/L: {pnl:.1f} pips<br>Operaciones: {count}" for hora, pnl, count in zip(hourly_data['horas'], hourly_data['pnl'], hourly_data['counts'])]
                    self.g_hourly_profile.add_bar(x=hourly_data['horas'], y=hourly_data['pnl'], marker=dict(color=colors, line=dict(color='#333', width=1)), hovertext=hover_texts, hoverinfo='text')
                    total_profit_blocks = sum(1 for pnl in hourly_data['pnl'] if pnl > 0)
                    total_loss_blocks = sum(1 for pnl in hourly_data['pnl'] if pnl < 0)
                    best_block_idx = np.argmax(hourly_data['pnl'])
                    worst_block_idx = np.argmin(hourly_data['pnl'])
                    best_block = hourly_data['horas'][best_block_idx]
                    worst_block = hourly_data['horas'][worst_block_idx]
                    best_pnl = hourly_data['pnl'][best_block_idx]
                    worst_pnl = hourly_data['pnl'][worst_block_idx]
                    titulo_perfil = f'📊 Perfil de Rentabilidad (bloques 30min): SL={stop}, TP={profit} ({direccion}) | ✅ {total_profit_blocks} bloques rentables | ❌ {total_loss_blocks} en pérdida | 🌟 Mejor: {best_block} ({best_pnl:.0f}p) | 💀 Peor: {worst_block} ({worst_pnl:.0f}p)'
                    self.g_hourly_profile.layout.title = titulo_perfil
            
            # Actualizar gráficos de DD
            print(f"📊 Actualizando gráficos de drawdowns...")
            self._actualizar_graficos_dd(dd_analysis)
            
            # Actualizar estadísticas principales
            print(f"📋 Actualizando estadísticas generales...")
            stats = curve_data['stats']
            filtros_activos = []
            if direccion != 'Ambas':
                filtros_activos.append(f'Dirección: {direccion}')
            if self.time_filter_enabled.value:
                inicio_min, fin_min = self.time_range_slider.value
                filtros_activos.append(f'Horario: {self._minutos_a_hora(inicio_min)} - {self._minutos_a_hora(fin_min)}')
            filtros_texto = ' | '.join(filtros_activos) if filtros_activos else 'Sin filtros'
            
            dd_info = ''
            if stats['max_dd'] > 0:
                if stats['dd_recovered']:
                    if stats['dd_start'] and stats['dd_end']:
                        tiempo_recuperacion = stats['dd_end'] - stats['dd_start']
                        dias = tiempo_recuperacion.days
                        horas = tiempo_recuperacion.seconds // 3600
                        if dias > 0:
                            dd_info = f' | ⏱️ Duración DD máx: {dias}d {horas}h'
                        else:
                            dd_info = f' | ⏱️ Duración DD máx: {horas}h'
                    else:
                        dd_info = f' | ⏱️ Duración DD máx: {stats["dd_duration"]} ops'
                else:
                    dd_info = f' | ⚠️ DD sin recuperar'
            
            self.stats_label.value = (
                f'<div style="background-color: #1a1a1a; padding: 12px; border-radius: 5px; margin-top: 10px; border-left: 3px solid #7C3AED;">'
                f'<span style="color: #e0e0e0;"><b>📊 Estadísticas Generales ({filtros_texto}):</b><br>'
                f'<div style="display: grid; grid-template-columns: 1fr 1fr; gap: 8px; margin-top: 8px;">'
                f'<div>✅ Ganadas: <span style="color: #00CF9B; font-weight: bold;">{stats["ganadas"]}</span> ({stats["win_rate"]:.1f}%)</div>'
                f'<div>❌ Perdidas: <span style="color: #FF6B35; font-weight: bold;">{stats["perdidas"]}</span></div>'
                f'<div>💰 Prom. Ganancia: <span style="color: #00CF9B;">{stats["avg_win"]:.1f}</span> pips</div>'
                f'<div>💸 Prom. Pérdida: <span style="color: #FF6B35;">{stats["avg_loss"]:.1f}</span> pips</div>'
                f'<div style="grid-column: 1 / -1;">📉 Max Drawdown: <span style="color: #DC2626; font-weight: bold;">{stats["max_dd"]:.1f}</span> pips{dd_info}</div>'
                f'</div></span></div>'
            )
            
            print(f"\n{'='*60}")
            print(f"✅ COMPLETADO - Análisis finalizado exitosamente")
            print(f"{'='*60}\n")
            
        except Exception as e:
            print(f"\n{'='*60}")
            print(f"❌ ERROR en _actualizar_curva_capital:")
            print(f"   Tipo: {type(e).__name__}")
            print(f"   Mensaje: {str(e)}")
            print(f"{'='*60}")
            import traceback
            traceback.print_exc()
            self.stats_label.value = f'<span style="color: #DC2626;">❌ Error: {str(e)}</span>'
    
    def _actualizar_graficos_dd(self, dd_analysis):
        """Actualiza los gráficos y estadísticas de análisis de drawdowns"""
        try:
            # Gráfico de Top 10 DDs
            with self.g_dd_top.batch_update():
                self.g_dd_top.data = []
                if dd_analysis['top_drawdowns']:
                    top_dds = dd_analysis['top_drawdowns']
                    profundidades = [dd['profundidad'] for dd in top_dds]
                    duraciones = [dd['duracion_ops'] for dd in top_dds]
                    rankings = list(range(1, len(top_dds) + 1))
                    
                    hover_texts = []
                    for i, dd in enumerate(top_dds):
                        texto = f"<b>DD #{i+1}</b><br>Profundidad: {dd['profundidad']:.1f} pips<br>Duración: {dd['duracion_ops']} ops"
                        if dd.get('tiempo_real'):
                            dias = dd['tiempo_real'].days
                            horas = dd['tiempo_real'].seconds // 3600
                            texto += f"<br>Tiempo: {dias}d {horas}h"
                        if not dd.get('recuperado', True):
                            texto += "<br><span style='color: #FF6B35;'>⚠️ Sin recuperar</span>"
                        hover_texts.append(texto)
                    
                    colors = ['#DC2626' if not dd.get('recuperado', True) else '#7C3AED' for dd in top_dds]
                    
                    self.g_dd_top.add_bar(
                        x=rankings,
                        y=profundidades,
                        marker=dict(color=colors, line=dict(color='#333', width=1)),
                        hovertext=hover_texts,
                        hoverinfo='text'
                    )
            
            # Gráfico de distribución
            with self.g_dd_dist.batch_update():
                self.g_dd_dist.data = []
                if dd_analysis['num_dds'] > 0 and dd_analysis['top_drawdowns']:
                    profundidades = [dd['profundidad'] for dd in dd_analysis['top_drawdowns']]
                    if len(profundidades) > 1:
                        self.g_dd_dist.add_histogram(
                            x=profundidades,
                            nbinsx=min(10, len(profundidades)),
                            marker=dict(color='#7C3AED', line=dict(color='#333', width=1))
                        )
            
            # Estadísticas textuales
            with self.dd_stats_output:
                clear_output(wait=True)
                
                freq_sig = dd_analysis['frecuencia_significativos']
                
                stats_html = (
                    f'<div style="background-color: #1a1a1a; padding: 12px; border-radius: 5px; border-left: 3px solid #DC2626;">'
                    f'<span style="color: #e0e0e0;"><b>📉 Análisis Detallado de Drawdowns:</b><br>'
                    f'<div style="margin-top: 8px; line-height: 1.8;">'
                    f'<div>🔢 Total de DDs: <span style="color: #fff; font-weight: bold;">{dd_analysis["num_dds"]}</span></div>'
                    f'<div>📊 DD Promedio: <span style="color: #FFA500; font-weight: bold;">{dd_analysis["dd_promedio"]:.1f}</span> pips</div>'
                    f'<div>💥 Total pips en DD: <span style="color: #DC2626; font-weight: bold;">{dd_analysis["dd_total_pips"]:.1f}</span> pips</div>'
                    f'<div>⏱️ Tiempo en DD: <span style="color: #FFD700;">{dd_analysis["porcentaje_en_dd"]:.1f}%</span> del total ({dd_analysis["tiempo_en_dd_ops"]} ops)</div>'
                    f'<div style="margin-top: 10px; padding-top: 10px; border-top: 1px solid #444;">'
                    f'<b>🚨 DDs Significativos (≥{Config.DD_THRESHOLD} pips):</b><br>'
                    f'<div style="margin-left: 15px; margin-top: 5px;">'
                    f'• Cantidad: <span style="color: #FF6B35; font-weight: bold;">{freq_sig["count"]}</span><br>'
                )
                
                if freq_sig['count'] > 1 and freq_sig.get('intervals'):
                    stats_html += (
                        f'• Frecuencia promedio: cada <span style="color: #00CF9B; font-weight: bold;">{freq_sig["avg_interval_days"]:.1f}</span> días<br>'
                        f'• Intervalo más corto: <span style="color: #FF6B35;">{min(freq_sig["intervals"]):.1f}</span> días<br>'
                        f'• Intervalo más largo: <span style="color: #00CF9B;">{max(freq_sig["intervals"]):.1f}</span> días'
                    )
                elif freq_sig['count'] == 1:
                    stats_html += '• Solo 1 evento significativo detectado'
                else:
                    stats_html += '• <span style="color: #00CF9B;">✓ Ningún evento significativo</span>'
                
                stats_html += '</div></div></div></span></div>'
                
                display(widgets.HTML(stats_html))
        except Exception as e:
            print(f"Error en actualización de gráficos DD: {str(e)}")
            import traceback
            traceback.print_exc()
    
        """Actualiza el gráfico de curva de capital y el perfil horario"""
        try:
            direccion = self.direction_selector.value
            hora_inicio = None
            hora_fin = None
            
            if self.time_filter_enabled.value:
                inicio_min, fin_min = self.time_range_slider.value
                hora_inicio = time(inicio_min // 60, inicio_min % 60)
                hora_fin = time(fin_min // 60, fin_min % 60) if fin_min < 1440 else time(23, 59)
            
            curve_data = simular_curva_capital(stop, profit, self.df_trades, direccion, hora_inicio, hora_fin)
            hourly_data = calcular_perfil_horario(stop, profit, self.df_trades, direccion)
            
            if curve_data['num_trades'] == 0:
                mensaje = f'⚠️ No hay operaciones'
                if direccion != 'Ambas':
                    mensaje += f' en {direccion}'
                if self.time_filter_enabled.value:
                    inicio_min, fin_min = self.time_range_slider.value
                    mensaje += f' entre {self._minutos_a_hora(inicio_min)} y {self._minutos_a_hora(fin_min)}'
                self.stats_label.value = f'<span style="color: #FF6B35;">{mensaje}</span>'
                with self.g_pnl_curve.batch_update():
                    self.g_pnl_curve.data = []
                    self.g_pnl_curve.layout.title = f'Sin operaciones para SL={stop}, TP={profit}'
                with self.g_hourly_profile.batch_update():
                    self.g_hourly_profile.data = []
                    self.g_hourly_profile.layout.title = 'Perfil de Rentabilidad - Sin datos'
                return
            
            with self.g_pnl_curve.batch_update():
                self.g_pnl_curve.data = []
                self.g_pnl_curve.add_scatter(
                    x=curve_data['x'],
                    y=curve_data['y'],
                    mode='lines',
                    line=dict(color=Config.CURVE_COLOR, width=2.5),
                    name=f'SL={stop}, TP={profit}'
                )
                final_pnl = curve_data['y'][-1]
                num_trades = curve_data['num_trades']
                direccion_emoji = {'Ambas': '↕️', 'Largo': '📈', 'Corto': '📉'}
                titulo = f'{direccion_emoji[direccion]} Curva de Capital: Stop={stop}, Profit={profit} | P/L Final: {final_pnl:.0f} pips | Operaciones: {num_trades}'
                if self.time_filter_enabled.value:
                    inicio_min, fin_min = self.time_range_slider.value
                    titulo += f' | 🕐 {self._minutos_a_hora(inicio_min)} - {self._minutos_a_hora(fin_min)}'
                self.g_pnl_curve.layout.title = titulo
            
            with self.g_hourly_profile.batch_update():
                self.g_hourly_profile.data = []
                if len(hourly_data['horas']) > 0:
                    colors = [Config.HOURLY_PROFIT_COLOR if pnl >= 0 else Config.HOURLY_LOSS_COLOR for pnl in hourly_data['pnl']]
                    hover_texts = [f"<b>{hora}</b><br>P/L: {pnl:.1f} pips<br>Operaciones: {count}" for hora, pnl, count in zip(hourly_data['horas'], hourly_data['pnl'], hourly_data['counts'])]
                    self.g_hourly_profile.add_bar(x=hourly_data['horas'], y=hourly_data['pnl'], marker=dict(color=colors, line=dict(color='#333', width=1)), hovertext=hover_texts, hoverinfo='text')
                    total_profit_blocks = sum(1 for pnl in hourly_data['pnl'] if pnl > 0)
                    total_loss_blocks = sum(1 for pnl in hourly_data['pnl'] if pnl < 0)
                    best_block_idx = np.argmax(hourly_data['pnl'])
                    worst_block_idx = np.argmin(hourly_data['pnl'])
                    best_block = hourly_data['horas'][best_block_idx]
                    worst_block = hourly_data['horas'][worst_block_idx]
                    best_pnl = hourly_data['pnl'][best_block_idx]
                    worst_pnl = hourly_data['pnl'][worst_block_idx]
                    titulo_perfil = f'📊 Perfil de Rentabilidad (bloques 30min): SL={stop}, TP={profit} ({direccion}) | ✅ {total_profit_blocks} bloques rentables | ❌ {total_loss_blocks} en pérdida | 🌟 Mejor: {best_block} ({best_pnl:.0f}p) | 💀 Peor: {worst_block} ({worst_pnl:.0f}p)'
                    self.g_hourly_profile.layout.title = titulo_perfil
            
            stats = curve_data['stats']
            filtros_activos = []
            if direccion != 'Ambas':
                filtros_activos.append(f'Dirección: {direccion}')
            if self.time_filter_enabled.value:
                inicio_min, fin_min = self.time_range_slider.value
                filtros_activos.append(f'Horario: {self._minutos_a_hora(inicio_min)} - {self._minutos_a_hora(fin_min)}')
            filtros_texto = ' | '.join(filtros_activos) if filtros_activos else 'Sin filtros'
            
            # Formatear información de duración del drawdown
            dd_info = ''
            if stats['max_dd'] > 0:
                if stats['dd_recovered']:
                    # Se recuperó
                    if stats['dd_start'] and stats['dd_end']:
                        tiempo_recuperacion = stats['dd_end'] - stats['dd_start']
                        dias = tiempo_recuperacion.days
                        horas = tiempo_recuperacion.seconds // 3600
                        if dias > 0:
                            dd_info = f' | ⏱️ Duración DD: {dias}d {horas}h ({stats["dd_duration"]} ops)'
                        else:
                            dd_info = f' | ⏱️ Duración DD: {horas}h ({stats["dd_duration"]} ops)'
                    else:
                        dd_info = f' | ⏱️ Duración DD: {stats["dd_duration"]} operaciones'
                else:
                    # No se ha recuperado
                    dd_info = f' | ⚠️ DD sin recuperar ({stats["dd_duration"]} ops desde inicio)'
            
            self.stats_label.value = (
                f'<div style="background-color: #1a1a1a; padding: 12px; border-radius: 5px; margin-top: 10px; border-left: 3px solid #7C3AED;">'
                f'<span style="color: #e0e0e0;"><b>📊 Estadísticas ({filtros_texto}):</b><br>'
                f'<div style="display: grid; grid-template-columns: 1fr 1fr; gap: 8px; margin-top: 8px;">'
                f'<div>✅ Ganadas: <span style="color: #00CF9B; font-weight: bold;">{stats["ganadas"]}</span> ({stats["win_rate"]:.1f}%)</div>'
                f'<div>❌ Perdidas: <span style="color: #FF6B35; font-weight: bold;">{stats["perdidas"]}</span></div>'
                f'<div>💰 Prom. Ganancia: <span style="color: #00CF9B;">{stats["avg_win"]:.1f}</span> pips</div>'
                f'<div>💸 Prom. Pérdida: <span style="color: #FF6B35;">{stats["avg_loss"]:.1f}</span> pips</div>'
                f'<div style="grid-column: 1 / -1;">📉 Max Drawdown: <span style="color: #DC2626; font-weight: bold;">{stats["max_dd"]:.1f}</span> pips{dd_info}</div>'
                f'</div></span></div>'
            )
        except Exception as e:
            print(f"Error: {str(e)}")
            import traceback
            traceback.print_exc()
            self.stats_label.value = f'<span style="color: #DC2626;">❌ Error: {str(e)}</span>'
    
    def construir_ui(self):
        """Construye y retorna la interfaz de usuario completa"""
        sorting_box = widgets.HBox([self.sort_column, self.sort_order])
        main_filters = [w for k, w in self.filter_widgets.items() if '_score' not in k.lower()]
        score_filters = [w for k, w in self.filter_widgets.items() if '_score' in k.lower()]
        filter_tabs = widgets.Tab(children=[widgets.VBox(main_filters), widgets.VBox(score_filters)])
        filter_tabs.set_title(0, 'Métricas Principales')
        filter_tabs.set_title(1, 'Scores Ponderados')
        scatter_box = widgets.HBox([self.g_scatter_pf, self.g_scatter_cs])
        graph_tabs = widgets.Tab(children=[scatter_box, self.g_heatmap])
        graph_tabs.set_title(0, 'Análisis de Dispersión')
        graph_tabs.set_title(1, 'Mapa de Calor')
        curve_controls = widgets.VBox([
            widgets.HTML("<div style='background-color: #2a2a2a; padding: 12px; border-radius: 8px; margin-bottom: 10px; border-left: 3px solid #7C3AED;'><span style='color: #c0c0c0; font-size: 0.95em;'>🎯 <b>Filtros de visualización de curva</b></span></div>"),
            widgets.HBox([self.direction_selector, widgets.VBox([self.time_filter_enabled, self.time_range_slider, self.time_range_label], layout={'margin': '0 0 0 20px'})])
        ])
        
        display(widgets.HTML("""
            <style>
                body, .jp-Notebook { background-color: #1a1a1a !important; }
                .widget-output, .output_area { background-color: #1a1a1a !important; }
                .widget-box, .widget-hbox, .widget-vbox, .jupyter-widgets, .jupyter-widget { background-color: #1a1a1a !important; }
                .widget-label, .widget-readout, label { color: #e0e0e0 !important; font-weight: 500 !important; }
                .widget-dropdown select, select { background-color: #2a2a2a !important; color: #e0e0e0 !important; border: 1px solid #555 !important; padding: 6px 10px !important; border-radius: 5px !important; }
                .widget-dropdown select:hover, select:hover { background-color: #333 !important; border-color: #7C3AED !important; }
                .widget-radio-box { background-color: transparent !important; }
                .widget-radio-box label { color: #e0e0e0 !important; padding: 5px 10px !important; }
                .widget-radio-box input[type="radio"] { accent-color: #7C3AED !important; }
                .widget-checkbox { background-color: transparent !important; }
                .widget-checkbox label { color: #e0e0e0 !important; }
                .widget-checkbox input[type="checkbox"] { accent-color: #7C3AED !important; width: 18px !important; height: 18px !important; }
                .widget-slider { background-color: transparent !important; }
                .widget-slider .noUi-target { background: #2a2a2a !important; border: 1px solid #444 !important; }
                .widget-slider .noUi-connect { background: linear-gradient(90deg, #7C3AED 0%, #9D6FFF 100%) !important; }
                .widget-slider .noUi-handle { background: #7C3AED !important; border: 2px solid #9D6FFF !important; box-shadow: 0 2px 8px rgba(124, 58, 237, 0.4) !important; }
                .widget-slider .noUi-handle:hover { background: #9D6FFF !important; }
                .widget-tab { background-color: #1a1a1a !important; }
                .widget-tab > .p-TabBar { background-color: #1a1a1a !important; }
                .widget-tab > .p-TabBar .p-TabBar-tab { background: #2a2a2a !important; color: #c0c0c0 !important; border: 1px solid #444 !important; border-bottom: none !important; margin-right: 2px !important; border-radius: 5px 5px 0 0 !important; }
                .widget-tab > .p-TabBar .p-TabBar-tab:hover { background: #333 !important; color: #ffffff !important; }
                .widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current { background: #3a3a4a !important; color: #ffffff !important; border-bottom: 3px solid #7C3AED !important; font-weight: bold !important; }
                .widget-tab > .p-TabBar-content { background-color: #1a1a1a !important; }
                .p-TabPanel { background-color: #1a1a1a !important; }
                h1, h2, h3, h4, h5, h6 { color: #ffffff !important; }
                p { color: #c0c0c0 !important; }
                hr { border-color: #444 !important; border-width: 1px !important; }
                .output_subarea { background-color: #1a1a1a !important; }
                .widget-text input { background-color: #2a2a2a !important; color: #e0e0e0 !important; border: 1px solid #555 !important; border-radius: 5px !important; padding: 6px 10px !important; }
                .widget-button, button { background-color: #7C3AED !important; color: #ffffff !important; border: none !important; border-radius: 5px !important; padding: 8px 16px !important; font-weight: 600 !important; }
                .widget-button:hover, button:hover { background-color: #9D6FFF !important; cursor: pointer !important; }
                ::-webkit-scrollbar { width: 10px; height: 10px; }
                ::-webkit-scrollbar-track { background: #2a2a2a; }
                ::-webkit-scrollbar-thumb { background: #7C3AED; border-radius: 5px; }
                ::-webkit-scrollbar-thumb:hover { background: #9D6FFF; }
                div[class*="widget"], div[class*="jupyter"] { background-color: transparent !important; }
                .output, .output_wrapper, .output_scroll { background-color: #1a1a1a !important; }
            </style>
        """))
        
        ui = widgets.VBox([
            widgets.HTML("<div style='background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); padding: 20px; border-radius: 10px; margin-bottom: 10px;'><h1 style='margin: 0; color: #ffffff;'>📊 Dashboard de Análisis de Estrategias</h1><p style='margin: 5px 0 0 0; color: #a0a0a0; font-size: 0.9em;'>Sistema avanzado de análisis cuantitativo de trading</p></div>"),
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #7C3AED 0%, #00CF9B 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<h3 style='color: #ffffff;'>🎛️ Panel de Control</h3>"),
            sorting_box,
            filter_tabs,
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #7C3AED 0%, #00CF9B 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<h3 style='color: #ffffff;'>📋 Top 10 Estrategias Filtradas</h3>"),
            self.table_output,
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #7C3AED 0%, #00CF9B 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<h3 style='color: #ffffff;'>📈 Visualizaciones Interactivas</h3>"),
            graph_tabs,
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #7C3AED 0%, #00CF9B 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<h3 style='color: #ffffff;'>💹 Análisis de Estrategia Seleccionada</h3><p style='color: #888; font-size: 0.9em;'>Haz clic en cualquier punto de los gráficos superiores para ver el análisis detallado</p>"),
            curve_controls,
            self.stats_label,
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #7C3AED 0%, #00CF9B 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<div style='margin: 15px 0;'><h4 style='color: #ffffff; margin-bottom: 5px;'>📉 Análisis Profundo de Drawdowns</h4><p style='color: #999; font-size: 0.85em; margin: 0;'>Visualiza todos los drawdowns, su frecuencia y comportamiento temporal</p></div>"),
            widgets.HBox([self.dd_stats_output, self.g_dd_top, self.g_dd_dist]),
            widgets.HTML("<div style='height: 2px; background: linear-gradient(90deg, #DC2626 0%, #7C3AED 100%); margin: 15px 0;'></div>"),
            widgets.HTML("<div style='margin: 15px 0;'><h4 style='color: #ffffff; margin-bottom: 5px;'>📊 Perfil de Rentabilidad por Bloques de 30 Minutos</h4><p style='color: #999; font-size: 0.85em; margin: 0;'>Identifica los bloques de tiempo más rentables y los \"bloques veneno\" de tu estrategia. <span style='color: #7C3AED; font-weight: bold;'>Este gráfico muestra TODAS las operaciones (sin filtro de horario)</span> para que veas el panorama completo.</p></div>"),
            self.g_hourly_profile,
            widgets.HTML("<div style='margin: 15px 0;'><h4 style='color: #ffffff; margin-bottom: 5px;'>💹 Curva de Capital Acumulada</h4><p style='color: #999; font-size: 0.85em; margin: 0;'>Visualiza la evolución del capital. <span style='color: #00CF9B; font-weight: bold;'>Usa el filtro de horario arriba</span> si quieres ver solo una sesión específica.</p></div>"),
            self.g_pnl_curve
        ])
        return ui

def main():
    """Función principal para inicializar el dashboard"""
    print("🚀 Iniciando Dashboard de Análisis de Estrategias v2.3...")
    print("-" * 60)
    df_results, df_trades = cargar_datos()
    if df_results is None or df_trades is None:
        print("\n❌ No se pudo inicializar el dashboard")
        return
    print("\n🔧 Construyendo interfaz...")
    dashboard = TradingDashboard(df_results, df_trades)
    ui = dashboard.construir_ui()
    display(ui)
    print("✓ Dashboard listo")
    print("-" * 60)
    print("💡 Tips:")
    print("   • Usa los sliders para filtrar estrategias")
    print("   • Haz clic en los gráficos para ver el análisis completo")
    print("   • El PERFIL HORARIO (30min) muestra TODOS los bloques sin filtro")
    print("   • La CURVA DE CAPITAL respeta el filtro de horario si lo activas")
    print("   • Busca 'bloques veneno' (barras rojas grandes) para evitarlos\n")
    dashboard._on_filter_change(None)

if __name__ == "__main__":
    main()

🚀 Iniciando Dashboard de Análisis de Estrategias v2.3...
------------------------------------------------------------
✓ Datos cargados: 37666 estrategias, 3790 operaciones

🔧 Construyendo interfaz...


HTML(value='\n            <style>\n                body, .jp-Notebook { background-color: #1a1a1a !important; …

✓ Dashboard listo
------------------------------------------------------------
💡 Tips:
   • Usa los sliders para filtrar estrategias
   • Haz clic en los gráficos para ver el análisis completo
   • El PERFIL HORARIO (30min) muestra TODOS los bloques sin filtro
   • La CURVA DE CAPITAL respeta el filtro de horario si lo activas
   • Busca 'bloques veneno' (barras rojas grandes) para evitarlos

